# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [6]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [8]:
import utils.config as cfg
all_config = cfg.get_config(True)
config = AttrDict(
    use_wandb       = all_config['embeddings']['use_wandb'],
    wandb_group     = all_config['embeddings']['wandb_group'],
    wandb_entity    = all_config['wandb']['user'],
    wandb_project   = all_config['wandb']['project'],
    enc_artifact    = all_config['embeddings']['train_artifact'],
    input_ar        = all_config['embeddings']['input_ar'],
    cpu             = all_config['embeddings']['cpu']
)

Current: /home/macu/work/nbs_pipeline
yml: ./config/base.yaml


## Run

In [ ]:
run = wandb.init(entity=config.wandb_entity,
                    project=config.wandb_project if config.use_wandb else 'work-nbs', 
                    group=config.wandb_group,
                    job_type='embeddings', 
                    mode='online' if config.use_wandb else 'disabled',
                    anonymous = 'never' if config.use_wandb else 'must',
                    config=config,
                    #id = 'embeddingsProvider',
                    resume='allow')

In [ ]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [ ]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [ ]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()
enc_learner

Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [ ]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [ ]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

In [ ]:
df = input_ar.to_df()
df

In [ ]:
df.shape

In [ ]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

In [ ]:
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

In [ ]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [ ]:
run.finish()